<a href="https://colab.research.google.com/github/AlexanderCoudijzer/BM25-VSM-Search-Engine/blob/main/Search_engine_v0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading testing
see https://ir-datasets.com/beir.html#beir/scifact  
and https://ir-datasets.com/python.html

##Data Loading

In [1]:
%%capture
!pip install --upgrade ir_datasets

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import ir_datasets

In [2]:
dataset = ir_datasets.load("beir/scifact/train")

print(dataset.docs_count(), 'documents with:', dataset.docs_cls().__annotations__)
print(dataset.queries_count(), 'queries with:', dataset.queries_cls().__annotations__)
print(dataset.qrels_count(), 'qrels with:',dataset.qrels_cls().__annotations__)

5183 documents with: OrderedDict([('doc_id', <class 'str'>), ('text', <class 'str'>), ('title', <class 'str'>)])
809 queries with: OrderedDict([('query_id', <class 'str'>), ('text', <class 'str'>)])
919 qrels with: OrderedDict([('query_id', <class 'str'>), ('doc_id', <class 'str'>), ('relevance', <class 'int'>), ('iteration', <class 'str'>)])


In [3]:
%%capture
#this actually downloads the data, muted to avoid clutter
for doc in dataset.docs_iter()[:1]: 
    None
for q in dataset.queries_iter():
    None
for qrel in dataset.qrels_iter():
    None

In [4]:
corpus=[]
for doc in dataset.docs_iter():
    corpus.append([doc[0],doc[1],doc[2]])

documents = [doc[1] for doc in corpus] # only taking the text for simplicity, but we'll need the doc_id for the evaluation

## Scifact initial test

In [5]:
vectorizer = CountVectorizer(stop_words='english', strip_accents='ascii')
documents_vectorized = vectorizer.fit_transform(documents)
vocabulary = vectorizer.get_feature_names_out()
dataframe = pd.DataFrame(documents_vectorized.toarray(), columns=vocabulary)
dataframe

,00,000,0000,00001,00002,000035,00004,00005,00006,00008,...,zygosity,zygote,zygotes,zygotic,zymography,zymosan,zyxin,zz,zzw,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Defining all the variables we need:
k_1 = 1.2
b = 0.8
dfs = (dataframe > 0).sum(axis=0) # doc frequency
N = dataframe.shape[0] # total number of docs
idfs = np.log10(N/dfs) # inverse doc frequency
#dls = [len(d.split(' ')) for d in documents] # considering all words in doc
dls = dataframe.sum(axis=1).tolist() # considering words minus stop words in doc (better option)
avgdl = np.mean(dls) # single value, mean doc length (minus stop words)

# Applying the BM25 formula:
numerator = np.array((k_1 + 1) * dataframe)
denominator = np.array(k_1 *((1 - b) + b * (dls / avgdl))).reshape(N,1) + np.array(dataframe)
BM25_tf = numerator / denominator
idfs = np.array(idfs)
BM25_score = BM25_tf * idfs

bm25_idf = pd.DataFrame(BM25_score, columns=vocabulary)
bm25_idf

,00,000,0000,00001,00002,000035,00004,00005,00006,00008,...,zygosity,zygote,zygotes,zygotic,zymography,zymosan,zyxin,zz,zzw,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
query_num = 1 #query number to test
queries = []
for q in dataset.queries_iter(): # reading in all the queries
    queries.append([q[0],q[1]])
print([queries[query_num][1]])

vectorizer_q = CountVectorizer(stop_words='english', strip_accents='ascii')
query_vectorized = vectorizer_q.fit_transform([queries[query_num][1]])
q_terms = vectorizer_q.get_feature_names_out()
q_terms

['1 in 5 million in UK have abnormal PrP positivity.']


array(['abnormal', 'million', 'positivity', 'prp', 'uk'], dtype=object)

In [8]:
q_terms = [term for term in q_terms if term in bm25_idf.columns]
q_terms_only_df = bm25_idf[q_terms]
score_q_d = q_terms_only_df.sum(axis=1)
print(q_terms_only_df.shape)
sorted(zip(documents,score_q_d.values), key = lambda tup:tup[1], reverse=True)[:10]

(5183, 5)


[('OBJECTIVES To carry out a further survey of archived appendix samples to understand better the differences between existing estimates of the prevalence of subclinical infection with prions after the bovine spongiform encephalopathy epizootic and to see whether a broader birth cohort was affected, and to understand better the implications for the management of blood and blood products and for the handling of surgical instruments. DESIGN Irreversibly unlinked and anonymised large scale survey of archived appendix samples. SETTING Archived appendix samples from the pathology departments of 41 UK hospitals participating in the earlier survey, and additional hospitals in regions with lower levels of participation in that survey. SAMPLE 32,441 archived appendix samples fixed in formalin and embedded in paraffin and tested for the presence of abnormal prion protein (PrP). RESULTS Of the 32,441 appendix samples 16 were positive for abnormal PrP, indicating an overall prevalence of 493 per m

In [9]:
qrels = []
for qrel in dataset.qrels_iter(): # reading in all the relevancy scores
    qrels.append(qrel)

## VSM

In [10]:
dfs = (dataframe > 0).sum(axis=0) # doc frequency
N = dataframe.shape[0] # total number of docs
idfs = np.log10(N/dfs) # inverse doc frequency
tfidf_frame = dataframe * idfs

In [42]:
query_num = 1 #query number to test
queries = []
for q in dataset.queries_iter(): # reading in all the queries
    queries.append([q[0],q[1]])
print([queries[query_num][1]])
q_vectorizer = CountVectorizer(stop_words='english', strip_accents='ascii')
q = q_vectorizer.fit_transform([queries[query_num][1]])
q_voc = q_vectorizer.get_feature_names_out()

q_vector =[]
for t in dataframe.columns:
    if t in q_voc: q_vector.append(1)
    else: q_vector.append(0)

['1 in 5 million in UK have abnormal PrP positivity.']


In [43]:
doc_vector = np.array(tfidf_frame)
q_vector = np.array(q_vector)
#doc_vector.shape, q_vector.shape


In [41]:
from numpy.linalg import norm
VSM_scores = np.dot(doc_vector, q_vector)/(norm(doc_vector)*norm(q_vector))
sorted(zip(documents,VSM_scores), key = lambda tup:tup[1], reverse=True)[:10]

[('BACKGROUND The Global Burden of Diseases, Injuries, and Risk Factors Study 2015 provides an up-to-date synthesis of the evidence for risk factor exposure and the attributable burden of disease. By providing national and subnational assessments spanning the past 25 years, this study can inform debates on the importance of addressing risks in context. METHODS We used the comparative risk assessment framework developed for previous iterations of the Global Burden of Disease Study to estimate attributable deaths, disability-adjusted life-years (DALYs), and trends in exposure by age group, sex, year, and geography for 79 behavioural, environmental and occupational, and metabolic risks or clusters of risks from 1990 to 2015. This study included 388 risk-outcome pairs that met World Cancer Research Fund-defined criteria for convincing or probable evidence. We extracted relative risk and exposure estimates from randomised controlled trials, cohorts, pooled cohorts, household surveys, census

## VSM combo test

In [45]:
## VSM retrieval
print([queries[query_num][1]])
q_vector

['1 in 5 million in UK have abnormal PrP positivity.']


array([0, 0, 0, ..., 0, 0, 0])

In [53]:
doc_vector = np.array(bm25_idf)

BM25_VSM_scores = np.dot(doc_vector, q_vector)/(norm(doc_vector)*norm(q_vector))
sorted(zip(documents,BM25_VSM_scores), key = lambda tup:tup[1], reverse=True)[:10]

[('OBJECTIVES To carry out a further survey of archived appendix samples to understand better the differences between existing estimates of the prevalence of subclinical infection with prions after the bovine spongiform encephalopathy epizootic and to see whether a broader birth cohort was affected, and to understand better the implications for the management of blood and blood products and for the handling of surgical instruments. DESIGN Irreversibly unlinked and anonymised large scale survey of archived appendix samples. SETTING Archived appendix samples from the pathology departments of 41 UK hospitals participating in the earlier survey, and additional hospitals in regions with lower levels of participation in that survey. SAMPLE 32,441 archived appendix samples fixed in formalin and embedded in paraffin and tested for the presence of abnormal prion protein (PrP). RESULTS Of the 32,441 appendix samples 16 were positive for abnormal PrP, indicating an overall prevalence of 493 per m

https://ai.googleblog.com/2018/05/advances-in-semantic-textual-similarity.html
https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder 